# 🔌 실시간 전류/전압 측정 및 CSV 저장
이 파일은 ZMPT101B 전압 센서와 ACS712 전류 센서를 이용해 **실시간으로 전압과 전류를 측정하고**, 그 값을 **CSV 파일로 저장하는 코드**입니다.
ADS1115를 통해 I2C로 아날로그 값을 읽으며, 다음과 같은 과정을 포함합니다:

- **센서 보정값 적용**
    - ZMPT101B: `offset = 2.5539`, `scale = 997.6`
    - ACS712: `offset = 2.5087`, `sensitivity = 0.0990`
- I2C 오류 처리: 센서 측정 중 발생할 수 있는 예외 처리 포함
- CSV 저장 포맷
    - 컬럼: `timestamp`, `v_raw`, `c_raw`, `voltage(V)`, `current(A)`, `label`
    - 저장 위치: ../data/arc/
    - 주기적 저장: 100개의 샘플 단위로 저장 및 출력

> 이 코드는 아크 생성 구간의 실시간 측정 데이터를 수집하는 데 사용됩니다.




In [ ]:
# 라이브러리 임포트
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import pandas as pd
import numpy as np

In [ ]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 2/3  # ±6.144V 범위 설정

# ADC 채널 설정
voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [ ]:
# 센서 보정값
ZMPT_offset = 2.5539
ZMPT_scale = 997.6
ACS712_offset = 2.5087
ACS712_sensitivity = 0.0990

# 데이터 저장용 변수
data = []
count = 0

In [ ]:
# 초기값 측정 (I2C 오류 처리 포함)
while True:
    try:
        prev_v_raw = voltage_ch.voltage  # 전압 초기값 측정
        prev_c_raw = current_ch.voltage  # 전류 초기값 측정
        break
    except OSError as e:
        print(f"[초기 I2C 오류] {e}")
        time.sleep(0.001163)

try:
    while True:
        now = time.time()  # 현재 시간 기록

        # 센서 데이터 읽기 (I2C 오류 처리)
        try:
            v_raw = voltage_ch.voltage
            c_raw = current_ch.voltage
            prev_v_raw = v_raw
            prev_c_raw = c_raw
        except OSError as e:
            print(f"[I2C 오류 발생] {e}")
            v_raw = prev_v_raw
            c_raw = prev_c_raw

        # 보정값 적용
        real_voltage = (v_raw - ZMPT_offset) * ZMPT_scale
        real_current = (c_raw - ACS712_offset) / ACS712_sensitivity

        # 데이터 저장
        data.append([now, v_raw, c_raw, real_voltage, real_current, 1])
        count += 1

        # 데이터 프레임 생성 및 CSV 저장 (100개 단위)
        if len(data) >= 100:
            df = pd.DataFrame(data, columns=[
                'timestamp', 'v_raw', 'c_raw', 'voltage(V)', 'current(A)', 'label'
            ])

            try:
                df.to_csv("../data/arc/arc_*sec_*ohm_*.csv", mode='a', index=False, header=False)
            except Exception as e:
                print(f"[파일 저장 오류] {e}")

            # 데이터 통계 출력
            print(f"[{count}] 데이터 통계:")
            print(f"V_raw: {df['v_raw'].mean():.3f}, C_raw: {df['c_raw'].mean():.3f}")
            print(f"Voltage(V): {df['voltage(V)'].mean():.2f}, Current(A): {df['current(A)'].mean():.2f}")
            print("-" * 60)
            data = []

        time.sleep(0.001163)


except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")

[100] V_raw: 2.548, C_raw: 2.565
      Voltage(V): -5.90, Current(A): 0.57
------------------------------------------------------------
[200] V_raw: 2.546, C_raw: 2.552
      Voltage(V): -8.09, Current(A): 0.43
------------------------------------------------------------
[300] V_raw: 2.537, C_raw: 2.553
      Voltage(V): -17.14, Current(A): 0.44
------------------------------------------------------------
[400] V_raw: 2.539, C_raw: 2.555
      Voltage(V): -14.80, Current(A): 0.46
------------------------------------------------------------
[500] V_raw: 2.535, C_raw: 2.550
      Voltage(V): -18.41, Current(A): 0.42
------------------------------------------------------------
[600] V_raw: 2.543, C_raw: 2.570
      Voltage(V): -11.25, Current(A): 0.61
------------------------------------------------------------
[700] V_raw: 2.542, C_raw: 2.556
      Voltage(V): -11.73, Current(A): 0.48
------------------------------------------------------------
[800] V_raw: 2.539, C_raw: 2.558
      Volt